# Parsing

In [ ]:
# Load the dissertations into into python. I'm using a pandas data frame object for convenience.

import csv
import pandas as pd

dissertations = pd.read_csv("dissertations.csv")
dissertations.head()

In [ ]:
# The authors are given in "last name, first name" format.

dissertations["author"].head()

In [ ]:
# Try to create a variable that contains the last name of every author. I'll get you started.
# This function will extract everything in a string up until the first space:

def getlastname(s):
    words = s.split()
    return words[0]

# We can then apply it to every string in the "author" column of our data frame

dissertations["author"].map(getlastname)



In [ ]:
# The resulting last names have trailing commas. Can you do better?
# Hint: The string function "find" returns the index of the first occurrence of its argument:

print("Apple".find("ppl"))
print("Banana".find("nan"))
print("Carrot".find("xy"))

# Hint 2: Use the python "slice" syntax to extract substrings from a string.

s = "apophenia"
s[1:4]

In [ ]:
# Can you write a function to extract the first name from the "author" field?
# Be careful: names in the author field can contain a middle initial, which follows the first name.

def getfirstname(s):
    # edit this function to return the first name
    return s

dissertations["author"].map(getfirstname)

# Regular Expressions

Sometimes we want to define more complex text searchers. Regular expressions are a mini-language for searching text. Regular expressions are *powerful*, but they quickly become *complex*. 

> Some people, when confronted with a problem, think "I know, I'll use regular expressions." Now they have two problems. 

> &mdash; Jamie Zawinski

Recall that we are trying to extract the first and last names from the "author" field:

In [ ]:
dissertations["author"].head()

Above, you used basic python functions to try to split the names into first and last name components. In this section we'll use regular expressions to accomplish this task.

In [ ]:
import re

In words, this is how I would explain where to find the last name component in the author field: "starting from the beginning of the line, take all the characters until you see a comma." We can build a regular expression that captures this idea from the following components:
* `^` Matches beginning of the line
* `.` Matches any character
* `+` A modifier that means "match one or more of the preceding expression"
The comma is not a special character in regular expressions, so it will match itself.

In [ ]:
m = re.search(r"^.+,", "Su, Yu-Wen")
m

Two things:
* We used "raw" string syntax `r""` to prevent python from escaping any special characters that might appear in our regular expression.
* If the regular expression matches something in the searched string, then `m` holds a python match-data object. Otherwise, it will be `None`.

To see the matched substring, use the `group` function.

In [ ]:
m.group()

This is almost what we want, but it would be nice if we weren't also capturing the comma. The problem is that we need the comma in our regular expression in order to know where the last name ends. The solution is to use regular expression groups by putting parentheses around parts of the regular expression that we want to refer to later.
* `(...)` Creates a group that can be referred to later.

In [ ]:
m = re.search(r"^(.*),", "Su, Yu-Wen")

print(m.group())
print(m.group(1))

In a match-data object `m`, `m.group(1)` will show you what was matched by the expression between the first set of parentheses,  `m.group(2)` will show you what was matched by the expression between the second set of parentheses, and so on. `m.group()` is a synonym for `m.group(0)` and it always refers to what was matched by the entire regular expression.

Now try to write a regular expression that will match the first name of the author (or to keep it simple, write a regular expression to match the first word after the comma). In addition to the bits of regular expressions we saw above, you might want to use one of the following:
* `\w` Matches a single alphanumeric character
* `\W` Matches anything that is not an alphanumeric character
* `\s` Matches a single whitespace character
* `\S` Matches anything that is not a whitespace character.

In [ ]:
regex = r"" # fill in your regular expression here
m = re.search(regex, "Su, Yu-Wen")

Does your regular expression work for all author names? The following code snipped will show you names that aren't matched by your regular expression.

In [ ]:
[name for name in dissertations["author"] if not re.search(regex, name)]

Regular expressions are a powerful tool and we're barely scratching the surface. 

## References
* Python documentation: https://docs.python.org/2/library/re.html#regular-expression-syntax
* Online regular expression tester (good for learning): http://regex101.com/

# String Comparators

Here is one possible solution to find the first word after the comma 

In [ ]:
# fwac stands for "first word after comma".
# In this example I compile my regular expression so that subsequent invocations are faster.
fwac = re.compile(r"^.*,\s?(\w+)")
fwac.search("Su, Yu-Wen").group(1)

# I will wrap my regular expression in a function that returns a blank string when no match is found.
# This allows me to handle malformed data. I also convert the names to lowercase so that I don't have
# to worry about letter case later on.

def getfirstname(s):
    m = fwac.search(s)
    if m:
        return m.group(1).lower()
    else:
        return ''
    
# Now add a "firstname" field to the dissertations data rame

dissertations["firstname"] = dissertations["author"].map(getfirstname)
dissertations["firstname"].head()

In [ ]:
# Let's add a "lastname" column while we're at it.

bc = re.compile(r"^([^,]*),")

def getlastname(s):
    m = bc.search(s)
    if m:
        return m.group(1).lower()
    else:
        return ''
    
dissertations["lastname"] = dissertations["author"].map(getlastname)
dissertations["lastname"].head()

In this section you can play around with some different string comparator algorithms which are provided by the [jellyfish](https://github.com/sunlightlabs/jellyfish) package.

In [ ]:
import jellyfish as jl

In [ ]:
# Create a set of all distinct first names. 

fns = set([s.lower() for s in dissertations["firstname"]])

Let's find which names in the data are most similar to yours using the Levenshtein distance (aka edit distance). First, type your name using all lowercase letters here:

In [ ]:
my_name = "joshua"

In the next step, we will define a higher-order function. This is a function that takes a function as a parameter and returns a new function as a result. This will be a little mind-bending if you haven't seen it before. I want you to define a function that:
* takes a string comparator function as an argument
* returns a function that takes a string as an argument, compares it to your name, and returns the result

In [ ]:
def my_name_comparator(string_comparator):
    def func(s):
        # compare s to my_name using string_comparator and return the result
        return
    
    return func

In [ ]:
# We can now define a function that will compare an arbitrary string to your name using edit distance and return the result.
# levenshtein_distance is the name of a function in the jellyfish package, which we aliased to jl

my_name_levenshtein_comparator = my_name_comparator(jl.levenshtein_distance)

In [ ]:
# Try it!
print(my_name_levenshtein_comparator("joshau"))
print(my_name_levenshtein_comparator("joseph"))
print(my_name_levenshtein_comparator("guiseppe"))

We now have all the pieces we need to:
* Take the set of unique first names we created above
* Create a list of (name, distance) tuples by comparing first names to your name using the my_name_levenshtein_comparator function
* Sort the list be increasing distance
* Print the top 20 result

In [ ]:
# What are the top 20 names closest to your name using Levenshtein distance?

Because we defined the `my_name_comparator` function above, we can easily repeat this exercise using any of the other string comparator functions in the jellyfish package. See how the top 20 names change if you use one of these distances instead:
* `jl.jaro_distance`
* `jl.jaro_winkler`
* `jl.damerau_levenshtein_distance`
* `jl.hamming_distance`

# Fellegi-Sunter Record Linkage

In this section we will "manually" perform the steps in Fellegi-Sunter record linkage. This is a demonstration and not a set of exercises (I don't leave pieces for you to fill in). My intention is to illustrate the Fellegi-Sunter algorithm by breaking it into bitesize pieces. 

In our example we will compare first names and last names using Levenshtein distance. In the Fellegi-Sunter algorithm, the result of a field comparison is assumed to follow a multinomial distribution. That means it can only take on finitely many values. Therefore we will define a function that compares to strings and returns the value 2, 1, or 0 to indicate an exact match, a nearly exact match, or anything else.

In [ ]:
def fuzzy_string_comparator(s1, s2):
    n = jl.levenshtein_distance(s1, s2)
    if n == 0:
        return 2
    if n <= 2:
        return 1
    else:
        return 0
    
print(fuzzy_string_comparator("joshua", "joshua"))
print(fuzzy_string_comparator("joshua", "joshau"))
print(fuzzy_string_comparator("joshua", "todd"))

The above function compares *fields* in a record. Next, we define a function that compares *records*. My record comparison function assumes that records will have the form of a tuple: (identifier, first name, last name). It returns a length 2 tuple that gives the result of applying our fuzzy string comparator to the first name and to the last name.

In [ ]:
def comparison_vector(rec1, rec2):
    """rec1 and rec2 have the form (id, first name, last name)"""
    m = fuzzy_string_comparator(rec1[1], rec2[1])
    n = fuzzy_string_comparator(rec1[2], rec2[2])
    return (m, n)

print(comparison_vector((1, "joshua", "tokle"), (2, "joshua", "smith")))
print(comparison_vector((3, "joshua", "tokle"), (4, "josue", "tolke")))

Next, we need to define m-weights and u-weights. An m-weight is the probability of seeing a particular field comparison outcome given that we are comparing two records that represent the same individual. A u-weight is the probability of seeing a particular field outcome given that we are comparing two records that do *not* represent the same individual. For example, if two records represent the same person, the first and last names should match with high probability (the m-weights for the comparison value 2 should be large). On the other hand, suppose we had month of birth in our data set. The probability that two random individuals will agree on month of birth is about 1/12, so we would assign a u-weight of about 1/12 to this outcome.

I'm going to make up some m-weights and u-weights here. In practice you might try to fit these to data or tweak them after seeing preliminary output. I don't claim that these are particularly good.

In [ ]:
# The outer list is length 2, corresponding to the number of fields we're comparing.
# Each inner list has length 3, because fuzzy string comparator returns 3 possible values.

m_weights = [[0.01, 0.14, 0.85], # m-weights corresponding to first name
             [0.01, 0.09, 0.90]] # m-weights corresponding to last name

u_weights = [[0.88, 0.10, 0.02], # u-weights corresponding to first name
             [0.91, 0.08, 0.01]] # u-weights corresponding to last name

Now we define a function that compares two records and returns the match score.

In [ ]:
import math

def match_score(rec1, rec2):
    m, n = comparison_vector(rec1, rec2)
    
    # what's the log-probability of seeing this comparison vector if the records are a match?
    log_pr_given_match = math.log(m_weights[0][m]) + math.log(m_weights[1][n])
    
    # what's the log-probability of seeing this comparison vector if the records are a nonmatch?
    log_pr_given_nonmatch = math.log(u_weights[0][m]) + math.log(u_weights[1][n])
    
    # return the log-likelihood-ratio
    return log_pr_given_match - log_pr_given_nonmatch

print(match_score((1, "joshua", "tokle"), (2, "joshua", "smith")))
print(match_score((1, "joshua", "tokle"), (4, "joshu", "tolke")))
print(match_score((1, "joshua", "tokle"), (7, "christina", "jones")))

Now let's try to link dissertation authors to source data. We will apply **blocking** to reduce the number of comparisons we perform. In particular, I am only going to compare graduate students from Purdue University whose last name begins with H. I then sort them by match score and print out those with a match score greater than 0. These are *potentially* matches.

In [ ]:
import MySQLdb

# Change this to match your credentials.

mydbuser = ""
mydbpass = ""

db = MySQLdb.connect("localhost", user=mydbuser, passwd=mydbpass)
cur = db.cursor()

cur.execute("""select researcher_id, lower(name_first), lower(name_last)
               from BigData.researcher
               where sourceid = 3
               and left(lower(name_last), 1) = 'h'""")

researchers = cur.fetchall()
researchers[:10]

In [ ]:
# This cell takes a minute to execute

potential_matches = []

for ix, row in dissertations.iterrows():
    if row["university"] == "Purdue" and row["lastname"][0] == "h":
        rec1 = (row["publication_number"], row["firstname"], row["lastname"])
        for rec2 in researchers:
            score = match_score(rec1, rec2)
            if score >= 0:
                potential_matches.append((score, rec1, rec2))
            
# Sort the output so the best matches appear at the top
potential_matches = sorted(potential_matches, key=lambda x: x[0], reverse=True)
potential_matches

# How did we do?